In [153]:
import os
os.chdir('/Users/sheldon/git/podly_app/new_files')

In [154]:
import glob
files = [f for f in os.listdir('.') if os.path.isfile(f)]
files = glob.glob('*.txt')


In [155]:
import pandas as pd
series = []
for i in files:
    series.append(i.split('.mp3')[0])

In [214]:
x = pd.DataFrame(series)
def read_podcast(file):
    tempFile = open(file).read()
    return tempFile

In [215]:
transcriptions = []
for i in files:
    transcription = open(i).read()
    transcriptions.append(transcription)
    

In [216]:
x['transcription'] = transcriptions

In [217]:
x.head()

,0,transcription
0,askmeanother-cameronesposito,support for the spot cast comes from being fun...
1,bullseye-olson-and-chang,and useful silent jesse whining you didn't get...
2,fivethirtyeightelections_2016-10-05-140700-800...,i'm having lived in the latest news of the sta...
3,freakonomics_podcast092816,freedom it's radio is supported by a concurren...
4,joerogan-p856,they were going to post on this episode of con...


In [218]:
x.columns = ['file','transcription']
fileList = range(1, len(files)+1)
x['index'] = fileList
x.head()

,file,transcription,index
0,askmeanother-cameronesposito,support for the spot cast comes from being fun...,1
1,bullseye-olson-and-chang,and useful silent jesse whining you didn't get...,2
2,fivethirtyeightelections_2016-10-05-140700-800...,i'm having lived in the latest news of the sta...,3
3,freakonomics_podcast092816,freedom it's radio is supported by a concurren...,4
4,joerogan-p856,they were going to post on this episode of con...,5


In [219]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [73]:
def tokenize_and_lower(textfile):
    tokens = word_tokenize(textfile)
    lower = [w.lower() for w in tokens]
    filtered_words = [word for word in lower if word not in stop]
    return filtered_words

from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import numpy as np
import nltk.data
from __future__ import division  # Python 2 users only
import nltk, re, pprint
from nltk import word_tokenize

In [213]:
tokenized_all_docs = [tokenize_and_lower(doc) for doc in transcriptions]
tokenized_all_docs

[['support',
  'spot',
  'cast',
  'comes',
  'funny',
  'productions',
  'presenting',
  'comedy',
  'bends',
  'first',
  'ever',
  'fully',
  'live',
  'stream',
  'stand',
  'comedy',
  'festival',
  'happening',
  'cobra',
  'twenty',
  'ninth',
  'listeners',
  'good',
  'twenty',
  'percent',
  'discount',
  'tickets',
  'going',
  'comedy',
  'binge',
  'coax',
  'last',
  'stream',
  'entering',
  'promo',
  'code',
  'another',
  'raymond',
  'ratio',
  'brilliant',
  'contestants',
  'chickens',
  'ah',
  'ha',
  'ha',
  'movement',
  'either',
  'win',
  'two',
  'hundred',
  'books',
  'ever',
  'wondered',
  'ah',
  'wallet',
  'couple',
  'times',
  'nothing',
  'says',
  'one',
  'window',
  'hunt',
  'book',
  'huh',
  'put',
  'like',
  'place',
  'price',
  'decreases',
  'time',
  "'ve",
  'know',
  'also',
  'became',
  'sane',
  'hot',
  'teacher',
  'eight',
  'sit',
  'ring',
  'music',
  "'re",
  'hot',
  'growth',
  "'s",
  'gon',
  'na',
  'end',
  'reference

In [75]:
from gensim import corpora

In [76]:
dictionary = corpora.Dictionary(tokenized_all_docs)

In [77]:
dictionary.save('sample.dict')

In [188]:
print dictionary

Dictionary(6806 unique tokens: [u'four', u'woods', u'verses', u'railing', u'woody']...)


In [191]:
corpus = [dictionary.doc2bow(text) for text in tokenized_all_docs]
corpora.MmCorpus.serialize('corpus.mm', corpus)

In [84]:
from gensim import models, corpora, similarities

In [207]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lsi = lsi[corpus_tfidf]
index = similarities.MatrixSimilarity(lsi[corpus])

In [208]:
lsi.print_topics(10)

[(0,
  u'0.135*"cash" + 0.112*"fire" + 0.098*"american" + 0.096*"morgan" + 0.086*"somalia" + 0.076*"correct" + 0.073*"shit" + 0.072*"embassy" + 0.072*"kenya" + 0.070*"winner"'),
 (1,
  u'-0.233*"democratic" + -0.233*"election" + 0.224*"fire" + -0.192*"muslim" + -0.133*"clinton" + -0.131*"immigrants" + -0.129*"americans" + -0.128*"voters" + -0.118*"industry" + -0.117*"demographics"'),
 (2,
  u'0.398*"cash" + 0.147*"sweden" + 0.125*"fire" + 0.122*"bills" + 0.121*"banks" + -0.103*"corley" + -0.103*"totems" + -0.103*"lectures" + 0.094*"billion" + -0.089*"courses"'),
 (3,
  u'-0.158*"fire" + 0.149*"morgan" + -0.131*"democratic" + -0.131*"election" + 0.119*"correct" + 0.109*"winner" + -0.107*"muslim" + 0.102*"performance" + -0.092*"voters" + 0.085*"character"'),
 (4,
  u'-0.177*"cash" + 0.173*"connection" + 0.151*"significant" + -0.113*"corley" + -0.113*"totems" + -0.113*"lectures" + 0.108*"patterns" + 0.108*"surprises" + -0.100*"courses" + 0.087*"return"'),
 (5,
  u'-0.276*"cash" + 0.245*"f

In [209]:
def get_related_podcasts_query(query):
    query = query.lower()
    vec_box = dictionary.doc2bow(query.split())
    vec_lsi = lsi[vec_box]
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])[0:10]
    related_df = pd.DataFrame(sims,columns=['index','score'])
    final_df = pd.merge(related_df, x, on='index')[['index','file','score']]
    return final_df

def get_related_podcasts(index):
    def getKey(item):
        return item[1]
    corpus = corpus_lsi[index]
    corpus = sorted(corpus, key=getKey, reverse=True)[0:10]
    related_df = pd.DataFrame(corpus,columns=['index','score'])
    final_df = pd.merge(related_df, x, on='index')[['index','file','score']]
    return final_df

In [221]:
get_related_podcasts_query('math')

,index,file,score
0,5,joerogan-p856,0.795419
1,8,THE_BLACK_TAPES_EPISODE_212_-_The_Axis_Mundi,0.614362
2,1,askmeanother-cameronesposito,0.084463
3,4,freakonomics_podcast092816,0.074925
4,9,this-american-life-abdi-560,0.064841
5,6,Lore-44fromwithin,0.064163
6,7,magificent_seven_and_fleabag,0.063497
7,3,fivethirtyeightelections_2016-10-05-140700-800...,0.052477
8,2,bullseye-olson-and-chang,0.048643


In [210]:
related_podcasts = list(get_related_podcasts(1)['index'])

In [211]:
def get_topics_per_podcast(podcast_index):
    def getKey(item):
        return item[1]
    topic_ids = [i for i in sorted(corpus_lsi[podcast_index], key=getKey, reverse=True) if i[1] > 0.10]
    def get_topic_arrays(topic_ids):
        x = []
        for id in topic_ids:
            list_of_words = sorted(lsi.show_topic(id[0], topn=5),key=getKey, reverse=True)
            z = []
            for word in list_of_words:
                if word[1] > .05:
                    z.append(word)
            x.append(z)
        return x
    topic_arrays = get_topic_arrays(topic_ids)
    return topic_arrays
related_podcasts_topics_words = [[related_podcasts[i],get_topics_per_podcast(related_podcasts[i])] for i in range(0, len(related_podcasts))]
episode_podcasts = list(get_related_podcasts(1)['file'])

In [212]:
for i,k in enumerate(related_podcasts_topics_words):
    print "Podcast: {}, ID: {}".format(i+1, k[0])
    print "Episode Title: {}".format(episode_podcasts[i])
    for num, topic in enumerate(k[1]):
        print "topic: {}".format(num)
        for word in topic:
            print "word: {}, score:{}".format(word[0], word[1])

Podcast: 1, ID: 7
Episode Title: magificent_seven_and_fleabag
topic: 0
word: cash, score:0.135160354963
word: fire, score:0.112313127804
word: american, score:0.0984519314402
word: morgan, score:0.0955392129237
word: somalia, score:0.085668201488
topic: 1
word: fire, score:0.224434183534
Podcast: 2, ID: 3
Episode Title: fivethirtyeightelections_2016-10-05-140700-8006-0-1-0.64k
topic: 0
word: cash, score:0.397700118953
word: sweden, score:0.147301438019
word: fire, score:0.124866497252
word: bills, score:0.121713914841
word: banks, score:0.121485976494
topic: 1
word: cash, score:0.135160354963
word: fire, score:0.112313127804
word: american, score:0.0984519314402
word: morgan, score:0.0955392129237
word: somalia, score:0.085668201488
topic: 2
word: morgan, score:0.148718513825
word: correct, score:0.11897481106
Podcast: 3, ID: 6
Episode Title: Lore-44fromwithin
topic: 0
word: morgan, score:0.148718513825
word: correct, score:0.11897481106
topic: 1
word: cash, score:0.135160354963
word: 